In [2]:
from vmlinux_to_elf.kallsyms_finder import KallsymsFinder, obtain_raw_kernel_from_file

path = '/Users/szhong/Downloads/bpf-study/btf/data/20.04-x86/tmp/vmlinux-5.4.0-170-generic'
# path = '/Users/szhong/Downloads/bpf-study/btf/data/20.04-x86/tmp/vmlinux-5.15.0-92-generic'
with open(path, 'rb') as f:
    kallsyms = KallsymsFinder(obtain_raw_kernel_from_file(f.read()))

In [14]:
from collections import defaultdict

from vmlinux_to_elf.kallsyms_finder import KallsymsSymbolType

# https://sourceware.org/binutils/docs/binutils/nm.html
ignored_sym_types = [
    KallsymsSymbolType.RODATA, # The symbol is in a read only data section.
    KallsymsSymbolType.DATA, # The symbol is in the initialized data section.
    KallsymsSymbolType.BSS, # The symbol is in the BSS data section.
    KallsymsSymbolType.ABSOLUTE, # The symbol’s value is absolute, and will not be changed by further linking.
]

fn_sym_types = [
    KallsymsSymbolType.TEXT, # The symbol is in the text (code) section.
    KallsymsSymbolType.WEAK_SYMBOL_WITH_DEFAULT, # The symbol is a weak symbol that has not been specifically tagged as a weak object symbol. 
    KallsymsSymbolType.WEAK_OBJECT_WITH_DEFAULT, # The symbol is a weak object. 
]

suffixes = [
    ".isra.0",
    ".part.0",
    ".constprop.0",
    ".cold",
]

counts = defaultdict(int)

for s in kallsyms.symbols:
    if s.symbol_type in ignored_sym_types:
        continue
    assert s.symbol_type in fn_sym_types, f"Unknown symbol type: {s.symbol_type}"
    if "." in s.name:
        if s.name.startswith("."):
            continue
        if s.name.split('.', 1)[-1].isdigit():
            continue
        match = False
        for suffix in suffixes:
            if s.name.endswith(suffix):
                counts[suffix] += 1
                match = True
        if not match:
            print(f"Unknown suffix: {s.name}")

counts

# cat /Users/szhong/Downloads/bpf-study/btf/data/20.04-x86/5.4.0-170-generic.txt | grep "FUNC '" | wc -l
for k, v in counts.items():
    print(f"{k}: {v / 48181 * 100:.2f}%")


.part.0: 1.77%
.isra.0: 1.62%
.constprop.0: 0.90%
.cold: 7.13%
